In [1]:
import os
import torchvision.transforms as transforms,datasets
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from facenet_pytorch import MTCNN
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torchvision.models as models
import numpy as np
import torch.nn.functional as F

from torch.utils.data import DataLoader, Dataset


In [2]:
if torch.cuda.is_available():
    print("✅ GPU доступен!")
    
    print(f"Имя устройства: {torch.cuda.get_device_name(0)}")
    print(f"Количество устройств: {torch.cuda.device_count()}")
    print(f"Текущий девайс: {torch.cuda.current_device()}")
else:
    print("❌ GPU не найден, используется CPU.")

✅ GPU доступен!
Имя устройства: NVIDIA GeForce RTX 3060
Количество устройств: 1
Текущий девайс: 0


In [3]:
print(torch.__version__)
print(torch.version.cuda)  # Должно быть не None
print(torch.backends.cudnn.version()) 

2.7.0+cu118
11.8
90100


In [4]:
class ImageDataset(Dataset):
    def __init__(self, image_dir, images_list, transform=None):
        self.image_dir = image_dir
        self.images_list = images_list
        self.transform = transform

    def __len__(self):
        return len(self.images_list)

    def __getitem__(self, idx):
        img_name = self.images_list[idx]
        img_path = os.path.join(self.image_dir, img_name)
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img

In [3]:

class SimpleAutoencoder(nn.Module):
    def __init__(self, latent_dim=128):
        super(SimpleAutoencoder, self).__init__()
        H = 200
        W = 170
        
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 4, stride=2, padding=1),  # B,32,H/2,W/2
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, stride=2, padding=1),  # B,64,H/4,W/4
            nn.ReLU(),
            nn.Conv2d(64, 128, 4, stride=2, padding=1), # B,128,H/8,W/8
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(128 * (H//8) * (W//8), latent_dim)  
        )
        
        
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128 * (H//8) * (W//8)),
            nn.Unflatten(1, (128, H//8, W//8)),
            nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1),  # B,64,H/4,W/4
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, stride=2, output_padding=1),   # B,32,H/2,W/2
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, 4, stride=2, output_padding=1),    # B,3,H,W
            nn.Sigmoid()  
        )
    
    def forward(self, x):
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        out = reconstructed[:, :, :x.size(2), :x.size(3)]

        return out,latent


In [4]:
def show_faces(original, reconstructed, n=5):

    fig, axes = plt.subplots(2, n, figsize=(15, 5))
    for i in range(n):
        axes[0, i].imshow(original[i].permute(1, 2, 0).clip(0, 1))
        axes[0, i].axis('off')
        axes[1, i].imshow(reconstructed[i].permute(1, 2, 0).clip(0, 1))
        axes[1, i].axis('off')
    axes[0, 0].set_ylabel('Оригинал', fontsize=14)
    axes[1, 0].set_ylabel('После A.E.', fontsize=14)
    plt.tight_layout()
    plt.show()

In [5]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((200, 170)),
    transforms.ToTensor()
])


autoencoder = SimpleAutoencoder().to(device)
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

image_dir = r'C:\Users\admin\.cache\kagglehub\datasets\jessicali9530\celeba-dataset\versions\2\img_align_celeba\img_align_celeba'
images = os.listdir(image_dir)


dataset = ImageDataset(image_dir, images, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)



NameError: name 'ImageDataset' is not defined

In [6]:
transform = transforms.Compose([
    transforms.Resize((200, 170)),
    transforms.ToTensor()
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

faces = []
image_dir = r'C:\Users\admin\.cache\kagglehub\datasets\jessicali9530\celeba-dataset\versions\2\img_align_celeba\img_align_celeba'
images = os.listdir(image_dir)

for img_name in tqdm(images[:50]):  # images — это список имён файлов
    img_path = os.path.join(image_dir, img_name)
    img = Image.open(img_path).convert('RGB')
    tensor_img = transform(img)
    faces.append(tensor_img)

faces = torch.stack(faces).to(device)


100%|██████████| 50/50 [00:00<00:00, 915.25it/s]


In [ ]:
mass_loss = []
# Тренируем

for epoch in range(10):
    loop = tqdm(dataloader, desc=f"Epoch {epoch+1}")
    for bath in loop:
        
        autoencoder.train()
        output,_ = autoencoder(bath)
        loss = loss_fn(output, bath)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        mass_loss.append(loss.item())
        loop.set_postfix(loss=loss.item())
        
    print(f"Эпоха {epoch+1}, Потери: {np.median(mass_loss):.4f}")

In [26]:
autoencoder.eval()
out,latent = autoencoder(faces)
reconstructed = out.detach().cpu()

show_faces(faces.cpu(), reconstructed)


In [32]:
latent = latent.detach().cpu().numpy()

In [35]:
latent.shape

(100, 128)

In [7]:
autoencoder = SimpleAutoencoder()
autoencoder.load_state_dict(torch.load("autoencoder_weights.pth"))
autoencoder.to(device)
autoencoder.eval()
out,latent = autoencoder(faces)
reconstructed = out.detach().cpu()
show_faces(faces.cpu(), reconstructed)